# Bedrock Knowledge Base Retrieval and Generation with Metadata Filtering

### Description:
This notebook demonstrates how to query and retrieve data from an Amazon Bedrock-powered knowledge base using different configurations, filters, and citation extraction. The steps include creating a query, retrieving responses, and printing the citations used for generating the results.


![Metadata Filtering](./metadata_filtering.png)

## 1. Load Configuration Variables

In [2]:
# Import utility functions
import advanced_rag_utils as aru

# Load configuration variables
variables = aru.load_variables("../variables.json")

# Display the loaded variables for confirmation
variables

{'accountNumber': '977018081517',
 'regionName': 'us-west-2',
 'collectionArn': 'arn:aws:aoss:us-west-2:977018081517:collection/x6j41w4wldt22s0mc4oa',
 'collectionId': 'x6j41w4wldt22s0mc4oa',
 'vectorIndexName': 'ws-index-',
 'bedrockExecutionRoleArn': 'arn:aws:iam::977018081517:role/advanced-rag-workshop-bedrock_execution_role-us-west-2',
 's3Bucket': '977018081517-us-west-2-advanced-rag-workshop',
 'kbFixedChunk': 'WFJRA66ARJ',
 'kbSemanticChunk': 'XRQUIKPQAB',
 'kbHierarchicalChunk': 'DBCDV6VOVB',
 'kbCustomChunk': 'SVUWGDQTQZ',
 'sagemakerLLMEndpoint': 'endpoint-llama-3-2-3b-instruct-2025-05-01-17-04-39'}

## 2. Set Up Required IDs and Model ARNs

In [3]:
# Setup IDs and ARNs
account_number = variables['accountNumber']
knowledge_base_id = variables['kbSemanticChunk']
model_id = 'us.amazon.nova-pro-v1:0'
model_arn = aru.get_model_arn(account_number, model_id)

## 3. Configure Bedrock Client

In [4]:
# Configure the Bedrock client
bedrock_agent_runtime = aru.setup_bedrock_client(variables["regionName"])

## 4. Define Query

In [5]:
# Define a simple query
query = "what was the % increase in sales?"

## 5. Retrieve Response Without Metadata Filter

In [6]:
# Retrieve response without metadata filter
response_without_metadata = aru.retrieve_and_generate_without_filter(
    query=query, 
    knowledge_base_id=knowledge_base_id, 
    model_arn=model_arn,
    bedrock_client=bedrock_agent_runtime
)

# Display the response
aru.display_rag_response(response_without_metadata)

The sales increased by 11% in 2024 compared to the prior year.


## 6. Retrieve and Print Citations Without Metadata Filter

In [7]:
# Extract citations used to generate the response
response_without_MD = aru.extract_citations(response_without_metadata)

# Print the citations
aru.citations_rag_print(response_without_MD)

# of citations or chunks used to generate the response:  1
Chunk 1:  Service sales primarily represent third-party seller fees, which includes commissions and any related fulfillment and shipping fees, AWS sales, advertising services, Amazon Prime membership fees, and certain digital media content subscriptions. Net sales information is as follows (in millions): Year Ended December 31, 
 
  2023 2024 
 
 Net Sales: North America $ 352,828 $ 387,497 International 131,200 142,906 AWS 90,757 107,556 
 
 Consolidated $ 574,785 $ 637,959 Year-over-year Percentage Growth: 
 
 North America 12 % 10 % International 11 9 AWS 13 19 
 
 Consolidated 12 11 Year-over-year Percentage Growth, excluding the effect of foreign exchange rates: 
 
 North America 12 % 10 % International 11 10 AWS 13 19 
 
 Consolidated 12 11 Net Sales Mix: 
 
 North America 61 % 61 % International 23 22 AWS 16 17 
 
 Consolidated 100 % 100 % 
 
 Sales increased 11% in 2024, compared to the prior year. Changes in foreign ex

## 7. Define Metadata Filter

The code below defines a metadata filter to narrow down the knowledge base search:
- Creates a complex filter using logical operators (andAll)
- The filter has two conditions that must both be true:
  1. docType must equal '10K Report'
  2. year must equal 2023
- This filter will limit retrieval to only chunks from 2023 10K reports
- The structure demonstrates how to build more complex queries with multiple conditions

This filter will be used to demonstrate selective retrieval from specific documents.

In [8]:
# Create a standard filter for 2023 10K Reports
one_group_filter = aru.create_standard_filter(docType='10K Report', year=2023)

## 8. Retrieve Response With Metadata Filter

In [9]:
# Retrieve response with metadata filter
response_with_metadata = aru.retrieve_and_generate_with_filter(
    query=query, 
    knowledge_base_id=knowledge_base_id, 
    model_arn=model_arn, 
    metadata_filter=one_group_filter,
    bedrock_client=bedrock_agent_runtime
)

# Display the response
aru.display_rag_response(response_with_metadata)

The percentage increase in sales was 9% in 2022 compared to the prior year.


## 9. Retrieve and Print Citations With Metadata Filter

In [10]:
# Extract and print citations for the filtered response
response_with_MD = aru.extract_citations(response_with_metadata)
aru.citations_rag_print(response_with_MD)

# of citations or chunks used to generate the response:  1
Chunk 1:  Net sales information is as follows (in millions): Year Ended December 31, 
 
  2021 2022 
 
 Net Sales: North America $ 279,833 $ 315,880 International 127,787 118,007 AWS 62,202 80,096 
 
 Consolidated $ 469,822 $ 513,983 Year-over-year Percentage Growth (Decline): 
 
 North America 18 % 13 % International 22 (8) AWS 37 29 
 
 Consolidated 22 9 Year-over-year Percentage Growth, excluding the effect of foreign exchange rates: 
 
 North America 18 % 13 % International 20 4 AWS 37 29 
 
 Consolidated 21 13 Net sales mix: 
 
 North America 60 % 61 % International 27 23 AWS 13 16 
 
 Consolidated 100 % 100 % 
 
 Sales increased 9% in 2022, compared to the prior year. Changes in foreign currency exchange rates reduced net sales by $15.5 billion in 2022. For a discussion of the effect of foreign exchange rates on sales growth, see “Effect of Foreign Exchange Rates” below. 
 
 North America sales increased 13% in 2022, comp

## 10. Advanced Metadata Filtering

Dynamically creating metadata filters allows  to create query-specific filters programmatically rather than hardcoding them.

We'll use a function that creates metadata filters programatically based on various parameters:
- company: Filter by company name
- year: Filter by year (can be a single year or list of years)
- docType: Filter by document type
- min_page/max_page: Filter by page number ranges
- s3_prefix: Filter by S3 URI prefix

The function builds a filter configuration based on the provided parameters,
combining them with appropriate operators (equals, greaterThanOrEquals, etc.).

In [11]:
# Compare growth rates across all Amazon business segments
query = "Compare the year-over-year growth rates for AWS, North America, and International segments, including factors that influenced performance differences"

# Use the query_financial_data function with dynamic filtering
response = aru.query_financial_data(
    query_text=query,
    kb_id=knowledge_base_id,
    model_arn=model_arn,
    bedrock_client=bedrock_agent_runtime,
    company="Amazon",
    year=[2023, 2024]
)

# Print the citations
aru.print_citations(response)

Using filter configuration:
{
  "andAll": [
    {
      "equals": {
        "key": "company",
        "value": "Amazon"
      }
    },
    {
      "orAll": [
        {
          "equals": {
            "key": "year",
            "value": 2023
          }
        },
        {
          "equals": {
            "key": "year",
            "value": 2024
          }
        }
      ]
    }
  ]
}

Generated Response:
The year-over-year growth rates for the segments are as follows:

- AWS: 13% in 2023 compared to 2022 - North America: Not explicitly provided in the results, but operating income changed from a loss of $2,847 million in 2022 to a profit of $14,877 million in 2023 - International: 11% in 2023 compared to 2022 Factors influencing performance differences:

- AWS: Growth was driven by increased customer usage, partially offset by pricing changes due to long-term customer contracts - North America: The significant change in operating income from a loss to a profit suggests a substant

In [12]:
# Filter for documents in a specific S3 folder
s3_prefix_2023 = f"s3://{variables['s3Bucket']}/pdf_documents/"
query = "What was the AWS revenue growth in 2023?"

# Use query_financial_data with S3 prefix filter
response_2023 = aru.query_financial_data(
    query_text=query,
    kb_id=knowledge_base_id,
    model_arn=model_arn,
    bedrock_client=bedrock_agent_runtime,
    year=[2023, 2024],
    s3_prefix=s3_prefix_2023
)

# Print the citations
aru.print_citations(response_2023)

Using filter configuration:
{
  "andAll": [
    {
      "orAll": [
        {
          "equals": {
            "key": "year",
            "value": 2023
          }
        },
        {
          "equals": {
            "key": "year",
            "value": 2024
          }
        }
      ]
    },
    {
      "startsWith": {
        "key": "x-amz-bedrock-kb-source-uri",
        "value": "s3://977018081517-us-west-2-advanced-rag-workshop/pdf_documents/"
      }
    }
  ]
}

Generated Response:
Sorry, I am unable to assist you with this request.

No citations available in the response.


### Dynamic Entity extraction to create filters on the fly
In the examples so far you knew the filters that you need to apply. Perhaps your application forces the user to pick a year or department name while asking questions. In those situations, the above approach would work.
However, you may have a situation where there is no way for a user to specify filters. Thus, the application may, at run time, have to figure out the filters based on a question.
For example, assume that your documents are stored in respective department folders such as HR, Finance, Legal, Science, Engineering, Customer Support, etc. 
Assume that your user asks an HR related question. There are two options for you.
### Option 1: 
You create a vector embedding for HR related questions and search the vector space in the entire knowledgebase. This will give you some context and might even pick up some HR related content from customer support content.
### Option 2: 
You ask an LLM to determine the topic to which the question most likely belongs to.Then you use the topic as a filter to query the knowledge base. This limits the search to fewer topics and hence reduces the noise from unrelated topics.
While this mightb improve accuracy because of richer context with reduced noise, this would also introduce extra costs because of an extra call to LLM to determine the topic to which the questuon belongs to.

In [13]:
# Define a sample query for entity extraction
user_prompt = "In Amazon's cash flow statement, what was the net income in years 2023 and 2024?"

# Extract entities (years and companies) from the query
years, companies = aru.extract_entities_from_query(
    user_prompt=user_prompt,
    model_id="anthropic.claude-3-5-haiku-20241022-v1:0",
    region_name=variables["regionName"]
)

# Display the extracted entities
print(f"Extracted years: {years}")
print(f"Extracted companies: {companies}")

Extracted years: [2023, 2024]
Extracted companies: ['Amazon']


In [14]:
# Use the extracted entities to filter a query
response = aru.query_financial_data(
    query_text=user_prompt,
    kb_id=knowledge_base_id,
    model_arn=model_arn,
    bedrock_client=bedrock_agent_runtime,
    company=companies,
    year=years
)

# Display the response with citations
aru.print_citations(response)

Using filter configuration:
{
  "andAll": [
    {
      "equals": {
        "key": "company",
        "value": "Amazon"
      }
    },
    {
      "orAll": [
        {
          "equals": {
            "key": "year",
            "value": 2023
          }
        },
        {
          "equals": {
            "key": "year",
            "value": 2024
          }
        }
      ]
    }
  ]
}

Generated Response:
The net income for Amazon in 2023 was $30,425 million. The model cannot find sufficient information to provide the net income for 2024.

Number of citations: 1

Citation 1:
----------------------------------------
Content: CONSOLIDATED STATEMENTS OF COMPREHENSIVE INCOME (LOSS) 
 
 (in millions) Year Ended December 31, 
 
  2021 2022 2023 
 
 Net income (loss) $ 33,364 $ (2,722) $ 30,425 Other comprehensive income (loss): 
 
 Foreign currency translation adjustments, net of tax of $47, $100, and $(55) (819) (2,586) 1,0...
Source: {'s3Location': {'uri': 's3://977018081517-us-west-2